In [3]:
#impoering the libraries and moduels
import torch
import torch.nn as nn
from torch.utils.data import Dataset ,DataLoader
from sklearn.model_selection import train_test_split
from torchsummary import summary
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'


#importing and cleaning the dataset
test_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\test_motion_data.csv")
train_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\train_motion_data.csv")
original_test_data_df = test_data_df.copy()
original_train_data_df = train_data_df.copy()
test_data_df.dropna(inplace=True)
train_data_df.dropna(inplace=True)


#normalizing 
for column in train_data_df:
    if train_data_df[column].dtype in  ['int','float']:
        train_data_df[column] = train_data_df[column]/train_data_df[column].abs().max()

for column in test_data_df:
    if test_data_df[column].dtype in ['int','float']:
        test_data_df[column] = test_data_df[column]/test_data_df[column].abs().max()

#omitting out the timestamp column cuz its irrelevant
test_data_df.drop(['Timestamp'], axis=1, inplace=True)
train_data_df.drop(['Timestamp'], axis=1, inplace=True)
print(train_data_df.head())

#replacing the class column with integeres witht he help of mapping
class_mapping = {
    'SLOW' : 0,
    'NORMAL' : 1,
    'AGGRESSIVE' : 2
}
train_data_df['Class'] = train_data_df['Class'].replace(class_mapping)
test_data_df['Class'] = test_data_df['Class'].replace(class_mapping)
print(train_data_df['Class'])



       AccX      AccY      AccZ     GyroX     GyroY     GyroZ   Class
0  0.000000  0.000000  0.000000  0.069951 -0.104000  0.082443  NORMAL
1 -0.325915 -0.230327 -0.028581 -0.033627  0.030545  0.109615  NORMAL
2 -0.119277 -0.026046  0.030865 -0.022838 -0.017455  0.071080  NORMAL
3  0.148124 -0.048610  0.093468  0.082179 -0.017818  0.044402  NORMAL
4  0.020407  0.165447 -0.009341  0.036145 -0.002182  0.044402  NORMAL
0       1
1       1
2       1
3       1
4       1
       ..
3639    0
3640    0
3641    0
3642    0
3643    0
Name: Class, Length: 3644, dtype: int64


C:\Users\HP\AppData\Local\Temp\ipykernel_46020\334160662.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data_df['Class'] = train_data_df['Class'].replace(class_mapping)
C:\Users\HP\AppData\Local\Temp\ipykernel_46020\334160662.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data_df['Class'] = test_data_df['Class'].replace(class_mapping)


## About the data ##
Looks like I was wrong to feed the data with basic processing as it is. This data shows the sequential information of a person/s driving the car, so it is better to take the average and 
standard deviation into accouint.
Potentially, I can also try this with a CNN + LSTM.

In [8]:
## Adding columns for mean and standard deviation across every 50 samples with rolling() from pandas
window = 50

features = train_data_df[['AccX','AccY','GyroX','GyroY','GyroZ']].rolling(window)

train_data_df['AccX_mean'] = features['AccX'].mean()
train_data_df['AccX_std'] = features['AccX'].std()
train_data_df['AccY_mean'] = features['AccY'].mean()
train_data_df['AccY_std'] = features['AccY'].std()
train_data_df['GyroX_mean'] = features['GyroX'].mean()
train_data_df['GyroX_std'] = features['GyroX'].std()
train_data_df['GyroY_mean'] = features['GyroY'].mean()
train_data_df['GyroY_std'] = features['GyroY'].std()
train_data_df['GyroZ_mean'] = features['GyroZ'].mean()
train_data_df['GyroZ_std'] = features['GyroZ'].std()
print(train_data_df.head())

       AccX      AccY      AccZ     GyroX     GyroY     GyroZ  Class  \
0  0.000000  0.000000  0.000000  0.069951 -0.104000  0.082443      1   
1 -0.325915 -0.230327 -0.028581 -0.033627  0.030545  0.109615      1   
2 -0.119277 -0.026046  0.030865 -0.022838 -0.017455  0.071080      1   
3  0.148124 -0.048610  0.093468  0.082179 -0.017818  0.044402      1   
4  0.020407  0.165447 -0.009341  0.036145 -0.002182  0.044402      1   

   AccX_mean  AccX_std  AccY_mean  AccY_std  GyroX_mean  GyroX_std  \
0        NaN       NaN        NaN       NaN         NaN        NaN   
1        NaN       NaN        NaN       NaN         NaN        NaN   
2        NaN       NaN        NaN       NaN         NaN        NaN   
3        NaN       NaN        NaN       NaN         NaN        NaN   
4        NaN       NaN        NaN       NaN         NaN        NaN   

   GyroY_mean  GyroY_std  GyroZ_mean  GyroZ_std  
0         NaN        NaN         NaN        NaN  
1         NaN        NaN         NaN        Na

## Current Workflow ##


1. Data cleaning - done
2. Dataset setup: most conceptual i believe
3. DataLoader magik
4. Design the model, ooh gawd
5. Training loop ooh lawd have mercy
6. Test the model, ** insert confused cat meme here **

In [5]:
###Dataset magik

#converting to numpy arrays
X = np.array(train_data_df.iloc[:,:-1])
Y = np.array(train_data_df.iloc[:,-1])
X_test = np.array(test_data_df.iloc[:,:-1])
Y_test = np.array(test_data_df.iloc[:,-1])

#creating a sweet validation set from train set
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.8)



In [6]:
##Apparently, the data is very sporadic with very less proportional changes happening across the axes.
##Implementing standardscaler from sklearn library,I have no idea what it is. I am just going to ask ai about this and figure it out.
##No code generation with AI though, that would defeat the purpose.
##here is to more focus on data pre processing cheerios!
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

ValueError: X has 6 features, but StandardScaler is expecting 16 features as input.

In [ ]:



class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    

In [ ]:
training_data = dataset(X_train, Y_train)
validation_data = dataset(X_val, Y_val)
test_data = dataset(X_test, Y_test)

#Loading the dataset into dataloader
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

In [ ]:
#Designing the model
class DriveModel(nn.Module):
    def __init__(self):
        super(DriveModel, self). __init__()
        #adding more widerlayers cuz the accuracy was shit 
        #adding dropout layers cuz the model aint stronk yet
        self.inputlayer = nn.Linear(X.shape[1], 128)
        self.reLu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.3)

        self.layer2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.3)

        self.layer3 = nn.Linear(64,32)
        self.relu3 = nn.ReLU()

        self.output_layer = nn.Linear(32,3)

    def forward(self, x):
        x = self.inputlayer(x)
        x = self.reLu1(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        output = self.output_layer(x)
        return output
    
model = DriveModel().to(device)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 1e-3, weight_decay= 1e-4)

In [ ]:
#Training loop
epochs = 80

for epoch in range(epochs):
    model.train()
    running_loss = 0
    running_accuracy = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        batch_loss = loss_fn(output, labels)
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()
        #accuracry counter
        _, predicted_classes = torch.max(output, 1)

        #counting the the predictions that match the labels
        running_accuracy += torch.sum(predicted_classes == labels).item()

    avg_train_loss = running_loss/len(train_dataloader)
    avg_train_acc = running_accuracy/len(train_dataloader.dataset)

    #Validation loop
    model.eval()
    val_running_loss = 0
    val_running_accuracy = 0
    with torch.no_grad():
        for val_input, val_labels in validation_dataloader:
            val_input = val_input.to(device)
            val_labels = val_labels.to(device)

            val_output = model(val_input)
            val_batch_loss = loss_fn(val_output, val_labels)

            val_running_loss += val_batch_loss.item()

            #accuracy for val set
            _, val_predicted_classes = torch.max(val_output, 1)
            val_running_accuracy += torch.sum(val_predicted_classes == val_labels).item()

    avg_val_loss = val_running_loss/len(validation_dataloader)
    avg_val_acc = val_running_accuracy/len(validation_dataloader.dataset)

    print('**' * 12)

    print(f"Epoch: [{epoch+1}/{epochs}]")
    print(f"Training loss(avg): {avg_train_loss} || Training acc(avg): {avg_train_acc:.4f}")
    print(f"Validation loss(avg): {avg_val_loss} || training acc(avg): {avg_val_acc:.4f}")




    


************************
Epoch: [1/80]
Training loss(avg): 1.092067061221763 || Training acc(avg): 0.3801
Validation loss(avg): 1.06953311096067 || training acc(avg): 0.4252
************************
Epoch: [2/80]
Training loss(avg): 1.066876772821766 || Training acc(avg): 0.4182
Validation loss(avg): 1.0660719644764196 || training acc(avg): 0.4129
************************
Epoch: [3/80]
Training loss(avg): 1.0584684396443302 || Training acc(avg): 0.4268
Validation loss(avg): 1.0529779484090598 || training acc(avg): 0.4348
************************
Epoch: [4/80]
Training loss(avg): 1.049496455225226 || Training acc(avg): 0.4364
Validation loss(avg): 1.0559558687002764 || training acc(avg): 0.4431
************************
Epoch: [5/80]
Training loss(avg): 1.0466925514887457 || Training acc(avg): 0.4391
Validation loss(avg): 1.0608365548693615 || training acc(avg): 0.4225
************************
Epoch: [6/80]
Training loss(avg): 1.0456776147019373 || Training acc(avg): 0.4398
Validation lo